### Neighborhoods In Toronto

By Kevin Wang

In [74]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from bs4 import BeautifulSoup
from urllib.request import urlopen

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

! pip install folium==0.5.0
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [75]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wikipage = urlopen(url).read().decode('utf-8')
result = BeautifulSoup(wikipage, 'html.parser')

wikitable = result.body.table.tbody

### Making Data Into DataFrame

In [76]:
def load_cell(element):
    cells = element.find_all('td')
    row = []
    for cell in cells:
        if cell.a:            
            if (cell.a.text):
                row.append(cell.a.text)
                continue
        row.append(cell.string.strip())
    return row

def load_row():    
    data = []  
    for tr in wikitable.find_all('tr'):
        row = load_cell(tr)
        if len(row) != 3:
            continue
        data.append(row)        
    return data

data = load_row()
columns = ['PostalCode', 'Borough', 'Neighborhood']
df = pd.DataFrame(data, columns=columns)
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [77]:
df.shape

(180, 3)

### Cleaning and Formatting Dataframe

In [78]:
df1 = df[df.Borough != 'Not assigned']
df1 = df1.sort_values(by=['PostalCode','Borough'])
df1.reset_index(inplace=True)
df1.drop('index',axis=1,inplace=True)
df1.head()


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [79]:
df1.shape

(103, 3)

In [80]:
df_postalCodes = df1['PostalCode']
df_postalCodes.drop_duplicates(inplace=True)
df2 = pd.DataFrame(df_postalCodes)
df2['Borough'] = '';
df2['Neighbourhood'] = '';


df2.reset_index(inplace=True)
df2.drop('index', axis=1, inplace=True)
df1.reset_index(inplace=True)
df1.drop('index', axis=1, inplace=True)

for i in df2.index:
    for j in df1.index:
        if df2.iloc[i, 0] == df1.iloc[j, 0]:
            df2.iloc[i, 1] = df1.iloc[j, 1]
            df2.iloc[i, 2] = df2.iloc[i, 2] + ',' + df1.iloc[j, 2]
            
for i in df2.index:
    s = df2.iloc[i, 2]
    if s[0] == ',':
        s = s [1:]
    df2.iloc[i, 2] = s

df2.head()


,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [81]:
df2.shape

(103, 3)

### Add Coordinates Data

In [82]:
df2['Latitude'] = '0';
df2['Longitude'] = '0';

df_coordinates = pd.read_csv('http://cocl.us/Geospatial_data')
df_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [83]:
for i in df2.index:
    for j in df_coordinates.index:
        if df2.iloc[i, 0] == df_geo.iloc[j, 0]:
            df2.iloc[i, 3] = df_geo.iloc[j, 1]
            df2.iloc[i, 4] = df_geo.iloc[j, 2]
            
df2.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.8067,-79.1944
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.7845,-79.1605
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7636,-79.1887
3,M1G,Scarborough,Woburn,43.771,-79.2169
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395


### Clustering

In [84]:
df3 = df2.copy()
df3 = df3[df2.Borough.str.contains("Toronto")]
df3.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.6764,-79.293
41,M4K,East Toronto,"The Danforth West, Riverdale",43.6796,-79.3522
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.669,-79.3156
43,M4M,East Toronto,Studio District,43.6595,-79.3409
44,M4N,Central Toronto,Lawrence Park,43.728,-79.3888


In [85]:
torontoMap = folium.Map(location=[43.65, -79.4], zoom_start=12)

X = df3['Latitude']
Y = df3['Longitude']
Z = np.stack((X, Y), axis=1)

kmCluster = KMeans(n_clusters=4, random_state=0).fit(Z)

clusters = kmCluster.labels_
colors = ['red', 'green', 'blue', 'yellow']
df3['Cluster'] = clusters

for latitude, longitude, borough, cluster in zip(X, Y, df3['Borough'], df3['Cluster']):
    label = folium.Popup(borough, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=8,
        popup=label,
        color='black',
        fill=True,
        fill_color=colors[cluster],
        fill_opacity=0.7).add_to(torontoMap) 
    
df3.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster
37,M4E,East Toronto,The Beaches,43.6764,-79.293,1
41,M4K,East Toronto,"The Danforth West, Riverdale",43.6796,-79.3522,1
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.669,-79.3156,1
43,M4M,East Toronto,Studio District,43.6595,-79.3409,1
44,M4N,Central Toronto,Lawrence Park,43.728,-79.3888,2


In [86]:
torontoMap